In [1]:
import pickle
from copy import deepcopy
from datetime import datetime as DT
from loguru import logger
import pandas as pd
from theano import shared
from pymc_models import PyMCModel
from pymc_models import bayes_nn_model_ARD_1HL_halfCauchy_hyperpriors as bnn_mdl

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


In [2]:
with open('../PickleJar/DataSets/AphiTrainTestSplitDataSets.pkl', 'rb') as fb:
    datadict = pickle.load(fb)
X_s_train = datadict['x_train_s']
y_train = datadict['y_train']
X_s_test = datadict['x_test_s']
y_test = datadict['y_test']

In [3]:
bands = [411, 443, 489, 510, 555, 670]
# create band-keyed dictionary to contain models
model_dict=dict.fromkeys(bands)

# create theano shared variable
X_shared = shared(X_s_train.values)
y_shared = shared(y_train['log10_aphy%d' % bands[0]].values)
# Fitting aphi411 model:
# Instantiate PyMC3 model with bnn likelihood
for band in bands:
    logger.info("processing aphi{band}", band=band)
    X_shared.set_value(X_s_train.values)
    y_shared.set_value(y_train['log10_aphy%d' % band].values)
    bnn = PyMCModel(bnn_mdl, X_shared, y_shared )
    bnn.model.name = 'bnn_aphy%d' %band
    bnn.fit(n_samples=2000, cores=4, chains=4, tune=10000,
                nuts_kwargs=dict(target_accept=0.95))
    ppc_train_ = bnn.predict(likelihood_name='likelihood')
    waic_train = bnn.get_waic()
    loo_train = bnn.get_loo()
    model_train = deepcopy(bnn.model)
    trace = deepcopy(bnn.trace_)
    run_dict = dict(model_train=model_train, trace=trace,
                    ppc_train=ppc_train_, loo_train=loo_train, waic_train=waic_train)
    X_shared.set_value(X_s_test.values)
    y_shared.set_value(y_test['log10_aphy%d' % band].values)
    model_test = deepcopy(bnn.model)
    ppc_test_ = bnn.predict(likelihood_name='likelihood')
    waic_test = bnn.get_waic()
    loo_test = bnn.get_loo()
    run_dict.update(dict(model_test=model_test, ppc_test=ppc_test_,
                         waic_test=waic_test, loo_test=loo_test))
    model_dict[band] = run_dict

2019-03-11 11:11:19.021 | INFO     | __main__:<module>:11 - processing aphi411
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, bias, w, lambda_m, sd_bias, tau]
Sampling 4 chains: 100%|██████████| 48000/48000 [00:56<00:00, 846.00draws/s] 
The number of effective samples is smaller than 25% for some parameters.
100%|██████████| 8000/8000 [00:54<00:00, 148.09it/s]
/accounts/ekarakoy/anaconda3/envs/bayesian_toa_project/lib/python3.7/site-packages/pymc3/stats.py:167: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.stack(logp)
/accounts/ekarakoy/anaconda3/envs/bayesian_toa_project/lib/python3.7/site-packages/pymc3/stats.py:218: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds

In [4]:
with open('../PickleJar/Results/hshoe_model_dict_%s.pkl' %DT.now(), 'wb') as fb:
        pickle.dump(model_dict, fb, protocol=pickle.HIGHEST_PROTOCOL)